### Spark Session Build and File Opening

In [1]:
file_path = "/home/jovyan/work/data/raw/farmers-protest-tweets-2021-2-4.json"

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FarmersProtestTweets").getOrCreate()

In [11]:
df = spark.read.json(file_path)

### Selecting data to be stored in GCS

In [4]:
df_transformed = df.select(['id', 'user.username', 'date', 'content'])

### Transforming date to yyyy-mm-dd

In [5]:
from pyspark.sql.functions import to_date


# creating formated date column
df_transformed = df_transformed.withColumn("date_transformed", to_date(df_transformed["date"]))
df_transformed = df_transformed.drop("date")

# renaming column "date_transformed" to "date"
df_transformed = df_transformed.withColumnRenamed("date_transformed", "date")



In [6]:
df_transformed.printSchema()

root
 |-- id: long (nullable = true)
 |-- username: string (nullable = true)
 |-- content: string (nullable = true)
 |-- date: date (nullable = true)



In [12]:
df_transformed_pandas = df_transformed.toPandas()

In [15]:
spark.stop()

In [13]:
import boto3
import json
from boto3.session import Session
from botocore.client import Config

In [16]:
#on secrets.json file

ACCESS_KEY = ""
SECRET_KEY = ""

In [26]:
session = Session(aws_access_key_id=ACCESS_KEY,
                  aws_secret_access_key=SECRET_KEY)

s3r = session.resource('s3', endpoint_url='https://storage.googleapis.com',
                      config=Config(signature_version='s3v4'))

client = session.client('s3', endpoint_url='https://storage.googleapis.com',
                      config=Config(signature_version='s3v4'))

In [27]:
bucket_name = 'latam-challenge-rtkseo-bucket'
object_key = 'staging/farmers_protest_tweets_staging.csv'
csv_data = df_transformed_pandas.to_csv(index=False, sep="~")

In [28]:
client.put_object(Bucket=bucket_name, Key=object_key, Body=csv_data)

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-guploader-uploadid': 'ABPtcPqiQCZAtwyM8Ob7ru_FgokzXUdGZj4ZnWZ1xXAz_BGBT38M1CLmx6eq6hJeDeNWEglnCAI',
   'etag': '"38f5ebfc5207d42203d8215a1db068ee"',
   'x-goog-generation': '1698863736420839',
   'x-goog-metageneration': '1',
   'x-goog-hash': 'crc32c=sn8MNQ==, md5=OPXr/FIH1CID2CFaHbBo7g==',
   'x-amz-checksum-crc32c': 'sn8MNQ==',
   'x-goog-stored-content-length': '26964400',
   'x-goog-stored-content-encoding': 'identity',
   'vary': 'Origin',
   'content-length': '0',
   'date': 'Wed, 01 Nov 2023 18:35:36 GMT',
   'server': 'UploadServer',
   'content-type': 'text/html; charset=UTF-8',
   'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'},
  'RetryAttempts': 0},
 'ETag': '"38f5ebfc5207d42203d8215a1db068ee"'}